# 평가

## 타이타닉 생존자 예측

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns

In [2]:
df = sns.load_dataset('titanic')
df.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


### 1. 데이터 전처리

- Feature selection

In [3]:
df = df[['survived','pclass','sex','age','sibsp','parch','fare','embarked','deck']]
df.head(3)

,survived,pclass,sex,age,sibsp,parch,fare,embarked,deck
0,0,3,male,22.0,1,0,7.2500,S,NaN
1,1,1,female,38.0,1,0,71.2833,C,C
2,1,3,female,26.0,0,0,7.9250,S,NaN


In [4]:
df.shape

(891, 9)

- X, y(survived) 파악

- 결측치 처리

In [5]:
df.isna().sum()

survived      0
pclass        0
sex           0
age         177
sibsp         0
parch         0
fare          0
embarked      2
deck        688
dtype: int64

In [6]:
# age - 평균으로 대체
df.age.fillna(df.age.mean(), inplace=True)
df.age.isna().sum()

0

In [7]:
# embarked - 범주형 - 최빈값으로 대체
df.embarked.value_counts()

S    644
C    168
Q     77
Name: embarked, dtype: int64

In [8]:
df.embarked.fillna('S', inplace=True)
df.embarked.isna().sum()

0

In [9]:
# deck - 열 삭제
df.drop(columns=['deck'], inplace=True)

In [10]:
df.isna().sum().sum()

0

- 카테고리 값(sex, embarked)을 숫자로 변환

In [11]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

In [12]:
df.sex = le.fit_transform(df.sex)
df.embarked = le.fit_transform(df.embarked)
df.head(3)

,survived,pclass,sex,age,sibsp,parch,fare,embarked
0,0,3,1,22.0,1,0,7.2500,2
1,1,1,0,38.0,1,0,71.2833,0
2,1,3,0,26.0,0,0,7.9250,2


### 2. Train/Test data set으로 분리

In [13]:
# X와 y를 넘파이 어레이로 변환
X = df.iloc[:, 1:].values
y = df.survived.values
X.shape, y.shape

((891, 7), (891,))

In [14]:
# y 값의 분포
np.unique(y, return_counts=True)    # df.survived.value_counts()

(array([0, 1]), array([549, 342]))

In [15]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, stratify=y, test_size=0.2, random_state=2022
)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((712, 7), (179, 7), (712,), (179,))

### 3. RandomForest 모델로 학습

In [16]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(random_state=2022)
rfc.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': 2022,
 'verbose': 0,
 'warm_start': False}

In [17]:
rfc.fit(X_train, y_train)

RandomForestClassifier(random_state=2022)

### 4. 평가

In [18]:
rfc.score(X_test, y_test)

0.8324022346368715

### 5. 3, 4 대신에 GridSearchCV 수행

In [19]:
params = {
    'max_depth': [2,4,6,8],
    'min_samples_split': [2,4,6]
}

In [20]:
from sklearn.model_selection import GridSearchCV
grid_rf = GridSearchCV(rfc, params, scoring='accuracy', cv=5)
grid_rf.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=RandomForestClassifier(random_state=2022),
             param_grid={'max_depth': [2, 4, 6, 8],
                         'min_samples_split': [2, 4, 6]},
             scoring='accuracy')

In [21]:
grid_rf.best_params_

{'max_depth': 4, 'min_samples_split': 4}

In [22]:
params = {
    'max_depth': [3,4,5],
    'min_samples_split': [3,4,5]
}
grid_rf = GridSearchCV(rfc, params, scoring='accuracy', cv=5)
%time grid_rf.fit(X_train, y_train)

CPU times: user 7.2 s, sys: 47.3 ms, total: 7.24 s
Wall time: 7.27 s


GridSearchCV(cv=5, estimator=RandomForestClassifier(random_state=2022),
             param_grid={'max_depth': [3, 4, 5],
                         'min_samples_split': [3, 4, 5]},
             scoring='accuracy')

In [23]:
grid_rf.best_params_

{'max_depth': 4, 'min_samples_split': 3}

In [24]:
best_rf = grid_rf.best_estimator_
best_rf.score(X_test, y_test)

0.8212290502793296

### 6. 엉터리 분류기

In [25]:
# 성별 생존률
df.groupby('sex')['survived'].mean()

sex
0    0.742038
1    0.188908
Name: survived, dtype: float64

In [26]:
df.groupby(['sex','pclass'])['survived'].mean()

sex  pclass
0    1         0.968085
     2         0.921053
     3         0.500000
1    1         0.368852
     2         0.157407
     3         0.135447
Name: survived, dtype: float64

In [27]:
from sklearn.base import BaseEstimator

class MyClassifier(BaseEstimator):
    # fit(), predict() 메쏘드만 재정의(Overriding)
    def fit(self, X, y):
        pass
    def predict(self, X):
        pred = np.zeros(X.shape[0], int)
        for i in range(X.shape[0]):
            if X[i,1] == 0:         # 여성이면
                pred[i] = 1         # 생존으로 처리
        return pred

In [28]:
my_clf = MyClassifier()
my_clf.fit(X_train, y_train)
pred_my = my_clf.predict(X_test)

In [29]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, pred_my)

0.7877094972067039

- 모델의 성능을 평가할 때 무조건적으로 정확도를 사용하는 것은 지양해야 함

In [30]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, pred_my)

array([[96, 14],
       [24, 45]])

In [32]:
confusion_matrix(y_test, best_rf.predict(X_test))

array([[103,   7],
       [ 25,  44]])

In [33]:
# MyClassifier
# 정밀도(Precision) = 45 / (14 + 45) = 0.7627
# 재현율(Recall) = 45 / (24 + 45) = 0.6522
from sklearn.metrics import precision_score, recall_score
precision_score(y_test, pred_my), recall_score(y_test, pred_my)

(0.7627118644067796, 0.6521739130434783)

In [34]:
# RFC
# 정밀도(Precision) = 44 / (7 + 44) = 0.8627
# 재현율(Recall) = 44 / (25 + 44) = 0.6377
pred_rf = best_rf.predict(X_test)
precision_score(y_test, pred_rf), recall_score(y_test, pred_rf)

(0.8627450980392157, 0.6376811594202898)

In [35]:
# F1 score(정밀도와 재현율의 조화 평균)
from sklearn.metrics import f1_score
f1_score(y_test, pred_my), f1_score(y_test, pred_rf)

(0.703125, 0.7333333333333333)

In [36]:
# AUC (Area under Curve(ROC))
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test, pred_my), roc_auc_score(y_test, pred_rf)

(0.7624505928853755, 0.787022397891963)